# Use batch deployments for image file processing with MLflow

The following notebook demonstrates how to use batch endpoints to deploy MLflow models that work with images. Particularly, we are going to deploy a TensorFlow model for the popular ImageNet classification problem. The MLflow models will be able to receive any image input and do all the require pre-post processing to generate the predictions in the shape and format needed.

This notebook requires:

- `tensorflow`
- `tensorflow_hub`
- `pillow`
- `azure-ai-ml`
- `azureml-mlflow`
- `pandas`
- `scipy`

## 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

### 1.1. Import the required libraries

In [ ]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import (
    BatchEndpoint,
    BatchDeployment,
    Model,
    AmlCompute,
    Data,
    BatchRetrySettings,
    CodeConfiguration,
    Environment,
)
from azure.ai.ml.constants import AssetTypes, BatchDeploymentOutputAction
from azure.identity import DefaultAzureCredential

### 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
# enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## 2. Using MLflow with images

When working with MLflow models that processes images, it is important to take into account that you won't be providing an scoring script. Hence, any data transformation that needs to be done before actually running the classifier needs to be done inside the model itself. Fortunately, you can design models that can compute these transformations:

### 2.1 Creating an MLflow model for image classification

The following example shows how to create a TensorFlow model that takes images of any size and preprocess them using keras layers.

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

model = tf.keras.Sequential(
    [
        tf.keras.layers.Resizing(
            244, 244, interpolation="bilinear", crop_to_aspect_ratio=False
        ),
        tf.keras.layers.Rescaling(1 / 255.0),
        hub.KerasLayer(
            "https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5"
        ),
        tf.keras.layers.Softmax(axis=-1),
    ]
)
model.build([None, None, None, 3])

Let's save this model in a local folder

In [ ]:
model_local_path = "imagenet-classifier-mlflow/model"
model.save(model_local_path)

### 2.2 Adding labels to the model predictions

We are going to include the labels for the predicted class in the directory so we can use them for inference:

In [ ]:
!wget https://azuremlexampledata.blob.core.windows.net/data/imagenet/ImageNetLabels.txt -P imagenet-classifier-mlflow/model

### 2.3 Creating a custom model loader for MLflow

Let's create a custom loader for the MLflow model:

In [ ]:
%%writefile imagenet-classifier-mlflow/code/module_loader.py

import pandas as pd
import tensorflow as tf

class TfClassifier():
    def __init__(self, model_path: str, labels_path: str):
        import numpy as np
        from tensorflow.keras.models import load_model
        
        self.model = load_model(model_path)
        self.imagenet_labels = np.array(open(labels_path).read().splitlines())

    def predict(self, data):

        preds = self.model.predict(data)

        pred_prob = tf.reduce_max(preds, axis=-1)
        pred_class = tf.argmax(preds, axis=-1)
        pred_label = [self.imagenet_labels[pred] for pred in pred_class]

        return pd.DataFrame({
            "class": pred_class, 
            "probability": pred_prob,
            "label": pred_label
        })

def _load_pyfunc(data_path: str):
    import os

    model_path = os.path.abspath(data_path)
    labels_path = os.path.join(model_path, "ImageNetLabels.txt")

    return TfClassifier(model_path, labels_path)

### 2.4 Adding a model signature for images

Indicating a signature for your model

In [ ]:
import numpy as np
import mlflow
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, TensorSpec

input_schema = Schema(
    [
        TensorSpec(np.dtype(np.uint8), (-1, -1, -1, 3)),
    ]
)
signature = ModelSignature(inputs=input_schema)

Creating the dependencies:

In [ ]:
from mlflow.utils.environment import _mlflow_conda_env

custom_env = _mlflow_conda_env(
    additional_conda_deps=None,
    additional_pip_deps=["tensorflow"],
    additional_conda_channels=None,
)

Logging the model:

In [ ]:
mlflow_model_path = "mlflow-model"
mlflow.pyfunc.save_model(
    mlflow_model_path,
    data_path="imagenet-classifier-mlflow/model",
    code_path=["imagenet-classifier-mlflow/code/module_loader.py"],
    loader_module="module_loader",
    conda_env=custom_env,
    signature=signature,
)

### 2.5 Registering the new model

In [ ]:
mlflow_model_name = f"{model_name}-mlflow"
ml_client.models.create_or_update(
    Model(
        name=mlflow_model_name,
        path=mlflow_model_path,
        type=AssetTypes.MLFLOW_MODEL,
    )
)

This new model can be used for batch scoring using batch deployments.

In [ ]:
model = ml_client.models.get(name=mlflow_model_name, label="latest")

## 3 Create Batch Endpoint

Batch endpoints are endpoints that are used batch inferencing on large volumes of data over a period of time. Batch endpoints receive pointers to data and run jobs asynchronously to process the data in parallel on compute clusters. Batch endpoints store outputs to a data store for further analysis.

To create an online endpoint we will use `BatchEndpoint`. This class allows user to configure the following key aspects:
- `name` - Name of the endpoint. Needs to be unique at the Azure region level
- `auth_mode` - The authentication method for the endpoint. Currently only Azure Active Directory (Azure AD) token-based (`aad_token`) authentication is supported. 
- `defaults` - Default settings for the endpoint.
   - `deployment_name` - Name of the deployment that will serve as the default deployment for the endpoint.
- `description`- Description of the endpoint.

### 3.1 Configure the endpoint

First, let's create the endpoint that is going to host the batch deployments. To ensure that our endpoint name is unique, let's create a random suffix to append to it. 

> In general, you won't need to use this technique but you will use more meaningful names. Please skip the following cell if your case:

In [ ]:
import random
import string

# Creating a unique endpoint name by including a random suffix
allowed_chars = string.ascii_lowercase + string.digits
endpoint_suffix = "".join(random.choice(allowed_chars) for x in range(5))
endpoint_name = "imagenet-classifier-" + endpoint_suffix

Let's configure the endpoint:

In [ ]:
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="An batch service to perform ImageNet image classification",
)

### 3.2 Create the endpoint
Using the `MLClient` created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.

In [ ]:
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

## 4. Create a batch deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing.

### 4.1 Creating an scoring script to work with the model

> Scoring scripts are not required for MLflow models.

### 4.2 Creating the compute

Batch deployments can run on any Azure ML compute that already exists in the workspace. That means that multiple batch deployments can share the same compute infrastructure. In this example, we are going to work on an AzureML compute cluster called `cpu-cluster`. Let's verify the compute exists on the workspace or create it otherwise.

In [ ]:
compute_name = "cpu-cluster"
if not any(filter(lambda m: m.name == compute_name, ml_client.compute.list())):
    print(f"Compute {compute_name} is not created. Creating...")
    compute_cluster = AmlCompute(
        name=compute_name, description="amlcompute", min_instances=0, max_instances=5
    )
    ml_client.begin_create_or_update(compute_cluster)

Compute may take time to be created. Let's wait for it:

In [ ]:
print("Waiting for compute", end="")
while ml_client.compute.get(name=compute_name).provisioning_state == "Creating":
    sleep(1)
    print(".", end="")

print(" [DONE]")

### 4.3 Creating the environment

Let's create the environment. In our case, our model runs on `Torch`. Azure Machine Learning already has an environment with the required software installed, so we can reutilize this environment.

> Environments are not required for MLflow models

## 3.1 Configure the deployment

We will create a deployment for our endpoint using the `BatchDeployment` class. This class allows user to configure the following key aspects.
- `name` - Name of the deployment.
- `endpoint_name` - Name of the endpoint to create the deployment under.
- `model` - The model to use for the deployment. This value can be either a reference to an existing versioned model in the workspace or an inline model specification.
- `compute` - Name of the compute target to execute the batch scoring jobs on
- `instance_count`- The number of nodes to use for each batch scoring job.		1
- `max_concurrency_per_instance`- The maximum number of parallel scoring_script runs per instance.
- `mini_batch_size`	- The number of files the code_configuration.scoring_script can process in one `run()` call.
- `retry_settings`- Retry settings for scoring each mini batch.		
   - `max_retries`- The maximum number of retries for a failed or timed-out mini batch (default is 3)
   - `timeout`- The timeout in seconds for scoring a mini batch (default is 30)
- `output_action`- Indicates how the output should be organized in the output file. Allowed values are `append_row` or `summary_only`. Default is `append_row`
- `output_file_name`- Name of the batch scoring output file. Default is `predictions.csv`
- `environment_variables`- Dictionary of environment variable name-value pairs to set for each batch scoring job.
- `logging_level`- The log verbosity level.	Allowed values are `warning`, `info`, `debug`. Default is `info`.

In [ ]:
mlflow_deployment = BatchDeployment(
    name="imagenet-classifier-rnet-mlflow",
    description="A ResNetV2 model architecture for performing ImageNet classification in batch stored in MLflow format",
    endpoint_name=endpoint.name,
    model=model,
    compute=compute_name,
    instance_count=2,
    max_concurrency_per_instance=1,
    mini_batch_size=10,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)

### 4.3 Create the deployment

Using the `MLClient` created earlier, we will now create the deployment in the workspace. This command will start the deployment creation and return a confirmation response while the deployment creation continues.

In [ ]:
ml_client.batch_deployments.begin_create_or_update(mlflow_deployment).result()

Let's update the default deployment name in the endpoint:

In [ ]:
endpoint = ml_client.batch_endpoints.get(endpoint_name)
endpoint.defaults.deployment_name = deployment.name
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

We can see the endpoint URL as follows:

In [ ]:
print(f"The default deployment is {endpoint.defaults.deployment_name}")

At this point, the deployment is ready to perform predictions.

# 4. Clean up Resources
Delete endpoint

In [ ]:
ml_client.batch_endpoints.begin_delete(name=endpoint_name)